### Import Dependencies

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

--------------------------------

# ABS_ERP_COMP

### Load CSV

In [2]:
csv_file = os.path.join('..', 'Resources', 'ABS_ERP_COMP_CLEAN.csv')
# csv_file = "Resources/ABS_ERP_COMP_CLEAN.csv"

erp_data_df = pd.read_csv(csv_file)
erp_data_df.head()

,DATAFLOW,MEASURE: Measure,REGION: Region,FREQ: Frequency,TIME_PERIOD: Time Period,OBS_VALUE,UNIT_MEASURE: Unit of Measure,UNIT_MULT: Unit of Multiplier,OBS_STATUS: Observation Status,OBS_COMMENT: Observation Comment
0,ABS:ERP_COMP_Q(1.0.0),Internal Arrivals,Australia,Q: Quarterly,1981-Q2,78.0,NUM: Number,3: Thousands,NaN,NaN
1,ABS:ERP_COMP_Q(1.0.0),Internal Departures,Australia,Q: Quarterly,1981-Q2,78.0,NUM: Number,3: Thousands,NaN,NaN
2,ABS:ERP_COMP_Q(1.0.0),Change Over Previous Quarter,Australian Capital Territory,Q: Quarterly,1981-Q2,NaN,NUM: Number,0: Units,u: not applicable,NaN
3,ABS:ERP_COMP_Q(1.0.0),Net Internal Migration,Australian Capital Territory,Q: Quarterly,1981-Q2,565.0,NUM: Number,0: Units,NaN,NaN
4,ABS:ERP_COMP_Q(1.0.0),Net Internal Migration,New South Wales,Q: Quarterly,1981-Q2,-6330.0,NUM: Number,0: Units,NaN,NaN


----------------------------------------

### Copy Region Colum to New DF - For Table 'States'

In [3]:
# Copy column to new DF
new_test_region_df = erp_data_df[['REGION: Region']].copy()
new_test_region_df.head()

,REGION: Region
0,Australia
1,Australia
2,Australian Capital Territory
3,Australian Capital Territory
4,New South Wales


In [4]:
# Drop duplicates
new_test_region_df = new_test_region_df.sort_values('REGION: Region').drop_duplicates('REGION: Region', keep='last')

In [5]:
# Create Index Column with Unique Values
new_test_region_df.reset_index(inplace=True)
new_test_region_df

,index,REGION: Region
0,303,Australia
1,553,Australian Capital Territory
2,2854,New South Wales
3,747,Northern Territory
4,2326,Queensland
5,2841,South Australia
6,2273,Tasmania
7,1287,Victoria
8,3114,Western Australia


In [6]:
# Rename Column Headers to match Schemata
new_test_region_df.rename(columns = {'index':'state_id', 'REGION: Region': 'state_name'}, inplace=True)
new_test_region_df

,state_id,state_name
0,303,Australia
1,553,Australian Capital Territory
2,2854,New South Wales
3,747,Northern Territory
4,2326,Queensland
5,2841,South Australia
6,2273,Tasmania
7,1287,Victoria
8,3114,Western Australia


### Copy Time Period Colum to New DF - For Table 'Quarters'

In [7]:
# Copy column to new DF
new_test_quarter_df = erp_data_df[['TIME_PERIOD: Time Period']].copy()
new_test_quarter_df.head()

,TIME_PERIOD: Time Period
0,1981-Q2
1,1981-Q2
2,1981-Q2
3,1981-Q2
4,1981-Q2


In [8]:
# Drop duplicates
new_test_quarter_df = new_test_quarter_df.sort_values('TIME_PERIOD: Time Period').drop_duplicates('TIME_PERIOD: Time Period', keep='last')

In [9]:
# Create Index Column with Unique Values
new_test_quarter_df.reset_index(level=0, inplace=True)
new_test_quarter_df

,index,TIME_PERIOD: Time Period
0,8,1981-Q2
1,25,1981-Q3
2,45,1981-Q4
3,59,1982-Q1
4,83,1982-Q2
...,...,...
159,3023,2021-Q1
160,3047,2021-Q2
161,3067,2021-Q3
162,3085,2021-Q4


In [10]:
# Rename Column Headers to match Schemata
new_test_quarter_df.rename(columns = {'index':'quarter_id', 'TIME_PERIOD: Time Period': 'date_quarter'}, inplace=True)
new_test_quarter_df

,quarter_id,date_quarter
0,8,1981-Q2
1,25,1981-Q3
2,45,1981-Q4
3,59,1982-Q1
4,83,1982-Q2
...,...,...
159,3023,2021-Q1
160,3047,2021-Q2
161,3067,2021-Q3
162,3085,2021-Q4


### Population_Change Table

In [11]:
# Copy column to new DF
population_df = erp_data_df[['TIME_PERIOD: Time Period', 'REGION: Region', 'OBS_VALUE']].copy()
population_df.head()

,TIME_PERIOD: Time Period,REGION: Region,OBS_VALUE
0,1981-Q2,Australia,78.0
1,1981-Q2,Australia,78.0
2,1981-Q2,Australian Capital Territory,NaN
3,1981-Q2,Australian Capital Territory,565.0
4,1981-Q2,New South Wales,-6330.0


In [12]:
# Change column headers to match shemata
population_df.rename(columns = {'REGION: Region':'state_id', 'TIME_PERIOD: Time Period': 'quarter_id', 'OBS_VALUE':'net_change'}, inplace=True)
population_df

,quarter_id,state_id,net_change
0,1981-Q2,Australia,78.0
1,1981-Q2,Australia,78.0
2,1981-Q2,Australian Capital Territory,NaN
3,1981-Q2,Australian Capital Territory,565.0
4,1981-Q2,New South Wales,-6330.0
...,...,...,...
3110,2022-Q1,Tasmania,1338.0
3111,2022-Q1,Victoria,33373.0
3112,2022-Q1,Victoria,-3350.0
3113,2022-Q1,Western Australia,2375.0


In [13]:
# Create Index Column with Unique Values
new_test_quarter_df.reset_index(level=0, inplace=True)
new_test_quarter_df

,index,quarter_id,date_quarter
0,0,8,1981-Q2
1,1,25,1981-Q3
2,2,45,1981-Q4
3,3,59,1982-Q1
4,4,83,1982-Q2
...,...,...,...
159,159,3023,2021-Q1
160,160,3047,2021-Q2
161,161,3067,2021-Q3
162,162,3085,2021-Q4


In [14]:
new_test_quarter_df.dtypes

index            int64
quarter_id       int64
date_quarter    object
dtype: object

In [15]:
population_df.dtypes

quarter_id     object
state_id       object
net_change    float64
dtype: object

In [16]:
# # convert_dict = {'index': object,
#                 'quarter_id': object 
#                 }
 
# new_test_quarter_df = new_test_quarter_df.astype(convert_dict)

In [17]:

# # Loop through the values in df1
# for index, row in new_test_region_df.iterrows():
#     # Check if the value in column 1 of df1 is present in column 1 of df2
#     if row['state_name'] in population_df['state_id'].values:
#         # Get the index of the matching row in df2
#         index2 = population_df[population_df['state_id'] == row['state_name']].index[-1]
#         # Replace the value in column 3 of df2 with the value in column 3 of df1
#         population_df.at[index2, 'state_id'] = row['state_id']

In [18]:

# # Loop through the values in df1
# for index, row in new_test_quarter_df.iterrows():
#     # Check if the value in column 1 of df1 is present in column 1 of df2
#     if row['date_quarter'] in population_df['quarter_id'].values:
#         # Get the index of the matching row in df2
#         index2 = population_df[population_df['quarter_id'] == row['date_quarter']].index[1]
#         # Replace the value in column 3 of df2 with the value in column 3 of df1
#         population_df.at[index2, 'quarter_id'] = row['quarter_id']

In [19]:
key_list = list(population_df['quarter_id'])
dict_lookup = dict(zip(new_test_quarter_df['date_quarter'], new_test_quarter_df['quarter_id']))
population_df['quarter_id'] = [dict_lookup[item] for item in key_list]
population_df.head()

,quarter_id,state_id,net_change
0,8,Australia,78.0
1,8,Australia,78.0
2,8,Australian Capital Territory,NaN
3,8,Australian Capital Territory,565.0
4,8,New South Wales,-6330.0


In [23]:
key_list = list(population_df['state_id'])
dict_lookup = dict(zip(new_test_region_df['state_name'], new_test_region_df['state_id']))
population_df['state_id'] = [dict_lookup[item] for item in key_list]
population_df.head()

,quarter_id,state_id,net_change
0,8,303,78.0
1,8,303,78.0
2,8,553,NaN
3,8,553,565.0
4,8,2854,-6330.0


In [24]:
# replace NaN with 0
population_df['net_change'] = population_df['net_change'].fillna(0)
population_df

,quarter_id,state_id,net_change
0,8,303,78.0
1,8,303,78.0
2,8,553,0.0
3,8,553,565.0
4,8,2854,-6330.0
...,...,...,...
3110,3114,2273,1338.0
3111,3114,1287,33373.0
3112,3114,1287,-3350.0
3113,3114,3114,2375.0


In [20]:
new_test_quarter_df

,index,quarter_id,date_quarter
0,0,8,1981-Q2
1,1,25,1981-Q3
2,2,45,1981-Q4
3,3,59,1982-Q1
4,4,83,1982-Q2
...,...,...,...
159,159,3023,2021-Q1
160,160,3047,2021-Q2
161,161,3067,2021-Q3
162,162,3085,2021-Q4


In [21]:
population_df.tail(20)

,quarter_id,state_id,net_change
3095,3085,Western Australia,8084.0
3096,3114,Australia,124.2
3097,3114,Australia,95.5
3098,3114,Australia,95.5
3099,3114,Australian Capital Territory,2545.0
3100,3114,Australian Capital Territory,-637.0
3101,3114,New South Wales,-9447.0
3102,3114,New South Wales,34685.0
3103,3114,Northern Territory,-600.0
3104,3114,Northern Territory,1053.0


In [22]:
# Test Output - DELETE
population_df.to_csv('../jon_clean_test.csv')